# 1. feladatsor - Asszinkron programozás
## Chat program szerver és kliens

Készíts egy Python chat alkalmazást asyncio segitségével. Legyen külon egy kliens és egy sever modul. Tetszőleges üzenettel lehessen bontani a kapcsolatot. A chat üzenetekete a szerver és a kliens is a saját szemszögéből jelenítse meg.

Ezeket a feladatokat nem tudod ebben a környezetben futtatni. A kódot külön fájlokban írd be saját környezetben, és vesd össze az itt található megoldással.

In [ ]:
import asyncio
import contextlib


async def serve(reader, writer):
    while True:
        try:
            print("Waiting for msg")
            request = await reader.readline()
            print("Got msg")
        except ConnectionRefusedError:
            print("Rekt")
            return

        if request == b'x\n':
            return
        else:
            print(f'client> {request.decode("utf-8")}')

            msg = input('server?> ')
            print(f'server> {msg}')
            msg = msg + '\n'
            writer.write(msg.encode)


async def launch(host, port):
    server = await asyncio.start_server(serve, host, port)
    await server.wait_closed()


main = asyncio.ensure_future(launch('127.0.0.1', 1337))
loop = asyncio.get_event_loop()

with contextlib.closing(loop):
    loop.run_until_complete(main)



In [ ]:
import asyncio
import contextlib


async def client(host, port):
    reader, writer = await asyncio.open_connection(host, port)
    while True:
        msg = input('client?> ')
        writer.write(msg.encode('utf-8'))
        response = await reader.readline()
        if response == b'x\n':
            return
        else:
            print(f'server> {response.decode("utf-8")}')


main = asyncio.ensure_future(client('127.0.0.1', 1337))
loop = asyncio.get_event_loop()

with contextlib.closing(loop):
    loop.run_until_complete(main)


# 2. feladatsor - RXPY reaktív programozás
## Tőzsdei vevő jelző

Készíts Python alkalmazást, ami az RxPy modul használatával valósít meg egy egyszerű tőzsdei vevő jelző rendszert.

Legyen egy listád a különböző részvényekről:
```python
stocks = [
{ 'TCKR' : 'APPL', 'PRICE': 200},
{ 'TCKR' : 'GOOG', 'PRICE': 90},
{ 'TCKR' : 'TSLA', 'PRICE': 120},
{ 'TCKR' : 'MSFT', 'PRICE': 150},
{ 'TCKR' : 'INTL', 'PRICE': 70},
]
```

Készíts ebből egy observable forrást. A forrás küldjön instrukciókat a feliratkozottaknak (observer függvény) de csak akkor ha a részvény árfolyam éppen 100 USD felett van.

Az observer függvényed és így a programod is ilyen kimenetet kell adjon:
```text
Received Instruction to buy APPL

Received Instruction to buy TSLA

Received Instruction to buy MSFT

All Buy Instructions have been received

```

In [4]:
import rx

stocks = [
{ 'TCKR' : 'APPL', 'PRICE': 200},
{ 'TCKR' : 'GOOG', 'PRICE': 90},
{ 'TCKR' : 'TSLA', 'PRICE': 120},
{ 'TCKR' : 'MSFT', 'PRICE': 150},
{ 'TCKR' : 'INTL', 'PRICE': 70},
]

def buy_stock_events(observer, scheduler):
    for stock in stocks:
        if stock['PRICE'] > 100:
            observer.on_next(stock['TCKR'])
    observer.on_completed()
    
source = rx.create(buy_stock_events)

source.subscribe(
    on_next=lambda value: print(f"Received instruction to buy {value}"),
    on_completed=lambda: print("All buy instructions have been received"),
    on_error=lambda error: print(f"Error occured {error}")
)

Received instruction to buy APPL
Received instruction to buy TSLA
Received instruction to buy MSFT
All buy instructions have been received


In [7]:
solution = %solution

assert_equal(solution, ['Received instruction to buy APPL', 'Received instruction to buy TSLA', 'Received instruction to buy MSFT', 'All buy instructions have been received'])